## A Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. for User X and User Y). Both, type and instance are stored and referenced with an ID (e.g. with a UUID) in the database.

This can support the deployment of chatbots in a web backend (state-less). For example, the UUIDs of the type and instance can be read as URL parameters from a URL that users have received from you.

A chatbot is created with the following arguments.
- database_file: File of SQLite (in Folder data/)
- type_id: Reference to chatbot type
- instance_id: Reference to chatbot instance (typically one per user - however, may also be shared by multiple users)
- type_role: Role of chatbot type (will be turned into a first prompt with role:system)
- instance_context: Context of chatbot instance (will be turned into a second prompt with role:system)
- instance_starter: Will be used to generate an initial message to the user (will be turned into a final prompt with role:system)

The following functions are meant to be used from an application (e.g. from controllers of a REST API).
- conversation_retrieve(with_system=False): Retrieve the previous conversation history (default: without prompts with role:system)
- start(): Returns an initial message to the user (Resulting from instance_starter prompt)
- respond(user_says): Returns an assistance response to user_says
- info_retrieve(): Returns the chatbot name, type role and instance context
- reset(): Resets the conversation so far

#### OpenAI API Key and Model

Create file chatbot/openai.py with the following content\
    OPENAI_KEY = "your OpenAI API key"\
    OPENAI_MODEL = "gpt-3.5-turbo-16k"\
(You may rename the file chatbot/openai_template.py to openai.py and set the keys there)

In [1]:
from chatbot.chatbot import Chatbot

#### Create a chatbot "Coach" for user X

In [2]:
bot_type = """
Als achtsamer und respektvoller Coach unterstützt du Personen dabei, ihre Steuererklärung unter Berücksichtigung aller Regelungen auszufüllen. 
Du hilfst ihnen, die notwendigen Dokumente wie Lohnabrechnungen und Bankauszüge bereitzulegen und informierst sie präzise, welche Informationen 
sie aus diesen Unterlagen in welche Abschnitte der Steuererklärung eintragen müssen. 
Du weist auf mögliche Abzüge hin und ermöglichst es ihnen, die Korrektheit ihrer Angaben eigenständig zu prüfen, während du sie darauf aufmerksam machst, 
dass eine Nachprüfung durch das Steueramt möglich ist. 
Dabei achtest du stets auf den Schutz ihrer persönlichen Daten und passt deine Beratung an ihre individuellen Bedürfnisse an. 
"""
bot_instance = """
In deiner Rolle als Coach unterstützt du nun im Rahmen einer Coaching-Session eine Person, deren Situation wie folgt beschrieben ist:
<ol>
    <li>Die Person wohnt in der Gemeinde Muster im Kanton Aargau, Schweiz.</li>
    <li>Sie hat ein Einkommen aus unselbstständiger Vollzeit-Tätigkeit als Büroangestellte/r.</li> 
    <li>Sie besitzt eine Eigentumswohnung mit einer Hypothek bei der Bank Swizzy, die sie selbst bewohnt.</li>
    <li>Sie besitzt Aktien von Nestlé, Roche und Swiss Re.</li>
    <li>Sie hat zwei betreuungspflichtige Kinder im Alter von drei Jahren.</li>
</ol>
Die Person soll in ihrer Steuererklärung folgende Angaben machen:
<ol>
    <li>Einkünfte aus unselbstständigem Erwerb</li>
    <li>betreuungspflichtige Kinder inklusive der Abzüge</li> 
    <li>Konto bei der Bank</li>
    <li>Aktien inklusive deren Steuerwerte</li>
</ol>
In dieser Coaching-Session sind keine weiteren Angaben erforderlich. 
Deine Aufgabe ist es, die Person zu befähigen, die relevanten Daten aus den Dokumenten Lohnausweis, Kontoauszug, Zinsauszug und Depotauszug korrekt in die Steuererklärung zu übertragen. 
Du hilfst die Person, die relevanten Abschnitte in den Dokumenten zu identifizieren. 
Ermutige die Person, die notwendigen Werte selbst zu ermitteln und verweise auf die entsprechenden Abschnitte in den Dokumenten, ohne konkrete Werte vorzugeben.
Dabei ist es wichtig, dass in dieser Sitzung keine anderen Angaben gemacht oder Dokumente hinzugezogen werden müssen.
Du gewährleistest den Schutz persönlicher Informationen, betonst die Bedeutung von Genauigkeit und Sorgfalt und förderst die Selbstständigkeit der Person.
"""
bot_starter = """
Bei deinen Antworten integriere Emojis, wo sie passend und hilfreich sind, um die Kommunikation freundlich und verständlich zu gestalten 😊. 
Achte darauf, dass die Antworten vollständig und präzis sind, ohne mit einem Doppelpunkt oder mit '... folgendes:' zu enden. 
Verwende <b><ol></b> oder <b><ul></b> mit <b><li></b>, um Informationen strukturiert in Listenform darzubieten, wenn dies die Klarheit fördert. 
Gliedere deine Antwort in mehrere <b><p></b>-Elemente, um die Lesbarkeit bei der Präsentation mehrerer Informationen oder Feedbacks zu erhöhen. 
Formatierung ist wichtig: Nutze gültiges HTML, wie <b><p></b> für Absätze und <b><b></b> für Hervorhebungen. 
Obwohl ein humorvoller Ton geschätzt wird, solltest du stets die Klarheit und Angemessenheit für den Kontext der Steuerberatung berücksichtigen. 
Deine Antworten sollten möglichst wenige Tokens beinhalten, wenn möglich weniger als 100, und in korrektem Deutsch verfasst sein.

Beginne das Gespräch mit einer freundlichen Begrüßung. Spreche die Person in einer geschlechtsneutralen und respektvollen Du-Form an.
Biete deine Hilfe an und warte auf eine konkrete Anfrage, bevor du umfassende Anleitungen gibst.
"""

In [3]:
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id="db4dfab7-0cf9-4916-adb2-5fdafb038070",
    user_id="e8950cdc-4d5a-4412-b3f1-f9f7357609b7",
    type_name="Coach",
    type_role=bot_type,
    instance_context=bot_instance,
    instance_starter=bot_starter
)

In [4]:
print(bot.start())

['<p>Herzlich willkommen zu unserer Steuer-Coaching Session! 😊 Ich freue mich, dir heute dabei zu helfen, wichtige Daten aus deinen Dokumenten korrekt in deine Steuererklärung einzutragen. Lass uns gerne sofort starten. Wo benötigst du momentan Hilfe oder welches Dokument möchtest du zuerst besprechen? </p>']


URL to be handed out to the user: If you are following the instructions to deploy your chatbot(s) to pythonanywhere, this is the URL to access your chatbot.

##### Generic URL
https://[your pythonanywhere user name].pythonanywhere.com/[type id]/[user_id]/chat
##### For Example
https://monkey23.pythonanywhere.com/053e97a0-6a91-4589-8602-340aa47b6376/7515865e-4097-4dd7-9567-d3c7a4c1ed07/chat

#### Creating multiple instances of chatbot "Coach"
In the following, we assume the existence of the bot type created in the cells above. We show example code that will generate N bot instances of that type. Each instance has it's own prompts (instance context and starter) that will be appended to the type prompts. Most importantly, each instance has its own chat history.

In [5]:
import uuid
import time

In [6]:
# Amount of instances to be created
number_of_instances = 12

# Change the following to a list of hardcoded instance IDs if you want to use existing users.
user_ids = [str(uuid.uuid4()) for _ in range(number_of_instances - 1)]

c  = 1 # counter for successful requests, don't change
error_c = 0 # counter for failed requests, don't change
for user_id in user_ids:
    bot = Chatbot(
        database_file="database/chatbot.db", 
        type_id="db4dfab7-0cf9-4916-adb2-5fdafb038070",
        user_id=user_id,
        instance_context=bot_instance,
        instance_starter=bot_starter
    )
    try:
        # each bot should have a first message to the user
        print(bot.start())
    except RuntimeError as error:
        print(error)
        error_c += 1
        continue
    c+=1
    time.sleep(15) #openai seems to produce more errors if we send the requests too fast.
    
print("successful: {}, failed: {}".format(c, error_c))


['<p>Hallo und herzlich willkommen zu unserer heutigen Coaching-Session zur Steuererklärung! 🙌 Aufgrund deiner Situation kann ich gut nachvollziehen, dass dieses Thema komplex erscheinen mag. Bitte sei versichert, dass wir gemeinsam Schritt für Schritt durchgehen, welche Angaben du in welchen Abschnitt deiner Steuererklärung eintragen musst. 😊</p>', '\n\n<p>Wir werden uns auf vier Hauptpunkte konzentrieren:\n  <ol>\n    <li>Deine Einkünfte aus unselbständiger Arbeit.</li>\n    <li>Deine Kinder und die damit verbundenen Abzüge.</li> \n    <li>Dein Konto bei der Bank.</li>\n    <li>Deine Aktien inklusive deren Steuerwerte.</li>\n  </ol>\nAuf Basis dieser Punkte und den vorhandenen Dokumenten; Lohnausweis, Kontoauszug, Zinsauszug und Depotauszug, werden wir die notwendige Info zusammenstellen, um deine Steuerklärung zu vervollständigen. 😊</p>\n\n', '<p>Bist du bereit loszulegen oder gibt es eine spezifische Frage oder einen Punkt, den du zuerst besprechen möchtest? 🧐</p>']
['<p>😊 Hallo! S

#### Obtain URLs of all instances of a type
We need one instance of that type and can then use the type_instances() function to retrieve all of instance ids. Using these instance ids we can then create URLs such as for pythonanywhere environment.

In [7]:
pythonanywhere_username = "a4e5bcd8"
type_id = "db4dfab7-0cf9-4916-adb2-5fdafb038070"
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id=type_id,
    user_id=user_ids[0]
)

for user_id in bot.type_instances():
    print("https://{}.pythonanywhere.com/{}/{}/chat".format(pythonanywhere_username, type_id, user_id))

https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/1fdba4b7-00d2-4e58-a9f4-268fee3df040/chat
https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/4b76f620-a51c-473b-a56f-a90fa51b6d8a/chat
https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/5d16b235-c899-4d89-b8e7-19737c20cf77/chat
https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/7dd76b75-1e91-4d93-9e20-edf1ac74a130/chat
https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/82ae84ab-2ed8-4d7e-8a0a-1ca06f2d2118/chat
https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/880b1a27-a973-497f-a9e2-73a38d414404/chat
https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/91a0426f-1a7c-48a3-9cc2-866b74495e02/chat
https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/98da4918-3b00-4d4e-bcbf-e023f6b168e2/chat
https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/a25782a